In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
import time
import os

from datetime import date
from datetime import timedelta
from datetime import datetime

import pickle

import json

import pandas as pd
import numpy as np
import random 


import pandas_ta as ta

import mplfinance as mpf 

In [3]:
from handle_data.read_data import load_candles

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [5]:
timeframe='1d'

experiment_nickname = "60d_to_halving"

In [6]:
symbols = [
    { 'symbol': 'BTC/USDT', 'asset_factor': 1_000_000, 'exchange':'Binance'},
    { 'symbol': 'ETH/USDT', 'asset_factor': 10_000, 'exchange':'Binance' },
    { 'symbol': 'XMR/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'MATIC/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'SOL/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'ADA/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'AVAX/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'RUNE/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'ENJ/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'XTZ/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'ENS/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    { 'symbol': 'FIL/USDT', 'asset_factor': 1_000, 'exchange':'Binance' },
    #{ 'symbol': 'HNT/USDT', 'asset_factor': 1_000, 'exchange':'KuCoin' },
]

In [7]:
history_days = 50 * 30
history_days = 90

start_date = (date.today() - timedelta(days=history_days)).strftime('%Y-%m-%d')
end_date=None

if experiment_nickname == "Up_and_down":
    start_date = "2019-11-01"
    end_date="2022-01-23"
elif experiment_nickname == "LongTerm":
    start_date = "2017-01-01"
    end_date=None
elif experiment_nickname == "Up":
    start_date = "2023-01-01"
    end_date=None
elif experiment_nickname == "Bear":
    start_date = "2021-11-12"
    end_date="2022-11-12"
elif experiment_nickname == "Range":
    start_date = "2022-06-16"
    end_date="2023-03-10"
elif experiment_nickname == "HNT":
    start_date = "2023-06-16"
    end_date=None
elif experiment_nickname == "Noise":
    start_date = "2023-12-15"
    end_date= "2024-01-29"
elif experiment_nickname == "Noise_short":
    start_date = "2024-01-13"
    end_date= "2024-01-18"
elif experiment_nickname == "Noise_1d":
    start_date = "2023-03-18"
    end_date= "2023-06-20"
elif experiment_nickname == "60d_to_halving": # https://halvingdates.com/crypto.php?sym=btc&name=Bitcoin
    start_date = "2020-03-11" # "2020-05-11"
    end_date= "2022-11-25"

In [8]:
print("start_date:",start_date)
print("end_date:",end_date)

start_date: 2020-03-11
end_date: 2022-11-25


In [9]:
if timeframe=='1m':
    length = 7
    multiplier = 1
elif timeframe=='15m':
    length = 7
    multiplier = 1
elif timeframe=='1h':
    length = 7
    multiplier = 1
elif timeframe=='1d':
    length = 34
    multiplier = 2
    #length = 13
    #multiplier = 1
    length = 3
    multiplier = 2
elif timeframe=='1w':
    length = 9
    multiplier = 1

In [10]:
for symbol in symbols:
    data = load_candles(symbol['exchange'], symbol['symbol'], timeframe=timeframe, start_date=start_date, end_date=end_date, )

    data["open_low_var"] = ((data["Open"] - data["Low"]) / data["Open"]) * 100
    open_low_var = data["open_low_var"].mean()

    data["dema"] = ta.dema(close = data.Close, length=length)
    data["stdev"] = ta.stdev(close = data.Close, length=length)

    data["dema_low"] = ta.dema(close = data.Low, length=length)
    data["stdev_low"] = ta.stdev(close = data.Low, length=length)

    data["upper"] = data["dema"] + (multiplier * data["stdev"])
    data["lower"] = data["dema"] - (multiplier * data["stdev"])

    data["lower_low"] = data["dema_low"] - (multiplier * data["stdev_low"])

    data["lower_open_low"] = data["lower"] - data["lower"] * ( open_low_var / 100 )

    apdict = [mpf.make_addplot(data['dema'],label="dema"),
              mpf.make_addplot(data['upper'],label="upper"),
              mpf.make_addplot(data['lower'],label="lower"),
              #mpf.make_addplot(data['lower_low'],label="lower_low"),
              mpf.make_addplot(data['lower_open_low'],label="lower_open_low"),]

    mpf.plot(data,type='candle', volume=True, show_nontrading=True,
             addplot=apdict,figratio=(30,20),figscale=2,
             savefig=f"_fv_{experiment_nickname}_{symbol['symbol'].replace('/','-')}_{timeframe}_plot.png")

reading /home/jovyan/work/data/BTC-USDT/1d/BTC-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/ETH-USDT/1d/ETH-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/XMR-USDT/1d/XMR-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/MATIC-USDT/1d/MATIC-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/SOL-USDT/1d/SOL-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/ADA-USDT/1d/ADA-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/AVAX-USDT/1d/AVAX-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/RUNE-USDT/1d/RUNE-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/ENJ-USDT/1d/ENJ-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/XTZ-USDT/1d/XTZ-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/ENS-USDT/1d/ENS-USDT_1d_Binance.pickle
reading /home/jovyan/work/data/FIL-USDT/1d/FIL-USDT_1d_Binance.pickle
